In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel, AutoProcessor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from PIL import Image
from transformers import TFAutoModel, AutoProcessor
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np
from PIL import Image

c:\Users\jasra\OneDrive\Desktop\MAJOR PROJECT\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load CSV files
train_df = pd.read_csv("C:\\Users\\jasra\\OneDrive\\Desktop\\MAJOR PROJECT\\MultiOFF_Dataset\\Split Dataset\\Training_meme_dataset.csv")
val_df = pd.read_csv("C:\\Users\\jasra\\OneDrive\\Desktop\\MAJOR PROJECT\\MultiOFF_Dataset\\Split Dataset\\Validation_meme_dataset.csv")
test_df = pd.read_csv("C:\\Users\\jasra\\OneDrive\\Desktop\\MAJOR PROJECT\\MultiOFF_Dataset\\Split Dataset\\Testing_meme_dataset.csv")

In [4]:
# Preprocess text using Hugging Face Tokenizer
text_model_name = "bert-base-uncased"
text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)

def preprocess_texts(texts, max_len=512):
    return text_tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

# Preprocess images using Hugging Face Processor
image_model_name = "google/vit-base-patch16-224-in21k"
image_processor = AutoProcessor.from_pretrained(image_model_name)

def preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        img = Image.open(image_path).convert("RGB")  # Convert to RGB
        img = img.resize((224, 224))  # Resize to 224x224
        img = np.array(img)  # Convert to NumPy array
        img = img / 255.0  # Normalize to [0, 1]
        images.append(img)
    return np.stack(images)



In [5]:
import re
from nltk.corpus import stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
EMAIL = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
NUMBERS_RE = re.compile(r'\d+')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):  #preprocessing of text 
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()
    text = EMAIL.sub('', text)
    text = NUMBERS_RE.sub('', text)  # Remove numbers
    text = REPLACE_BY_SPACE_RE.sub(' ',text)
    text = BAD_SYMBOLS_RE.sub('',text)    
    # text = text.replace('x','')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    
    return text


In [6]:
train_df['sentence'] = train_df['sentence'].apply(clean_text)
train_df['sentence'] = train_df['sentence'].apply(clean_text)
train_df['sentence'] = train_df['sentence'].apply(clean_text)

In [7]:
# Prepare train data
train_texts = preprocess_texts(train_df["sentence"])
train_images = preprocess_images([f"C:/Users/jasra/OneDrive/Desktop/MAJOR PROJECT/MultiOFF_Dataset/Labelled Images/{img}" for img in train_df["image_name"]])
train_df["label"] = train_df["label"].map({"Non-offensiv": 0, "offensive": 1})
train_labels = train_df["label"].values  # Convert to a NumPy array

# Prepare validation data
val_texts = preprocess_texts(val_df["sentence"])
val_images = preprocess_images([f"C:/Users/jasra/OneDrive/Desktop/MAJOR PROJECT/MultiOFF_Dataset/Labelled Images/{img}" for img in val_df["image_name"]])
val_df["label"] = val_df["label"].map({"Non-offensiv": 0, "offensive": 1})
val_labels = val_df["label"].values  # Convert to a NumPy array



In [8]:
print(train_images.shape)  # Should output: (batch_size, 224, 224, 3)

(445, 224, 224, 3)


In [6]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# EfficientNetV2 model for image feature extraction
def create_image_model():
    base_model = EfficientNetV2B0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model
    inputs = Input(shape=(224, 224, 3), name="image_input")
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)  # Reduce to a single feature vector
    x = Dense(128, activation="relu")(x)  # Add a dense layer for embedding
    return Model(inputs, x, name="ImageModel")

In [7]:
image_model = create_image_model()
image_model.summary()

Model: "ImageModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
Total params: 6,083,280
Trainable params: 163,968
Non-trainable params: 5,919,312
_________________________________________________________________


In [8]:
image_input = Input(shape=(224, 224, 3), name="image_input")
image_out = image_model(image_input)
image_features = Dense(1, activation="sigmoid", name="output")(image_out)

In [9]:
# Build Text Model
text_input = Input(shape=(512,), dtype=tf.int32, name="text_input")
text_mask = Input(shape=(512,), dtype=tf.int32, name="text_mask")
text_model = TFAutoModel.from_pretrained(text_model_name)
text_output = text_model(text_input, attention_mask=text_mask).pooler_output
text_features = Dense(128, activation="relu")(text_output)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [10]:
# Combine Features
combined_features = Concatenate()([text_features, image_features])
output = Dense(1, activation="sigmoid")(combined_features)

# Compile the Model
model = Model(inputs=[image_input, text_input, text_mask], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



In [ ]:
# Train the Model
history = model.fit(
    [train_images, train_texts["input_ids"], train_texts["attention_mask"]],
    train_labels,
    validation_data=(
        [val_images, val_texts["input_ids"], val_texts["attention_mask"]],
        val_labels
    ),
    epochs=4,
    batch_size=32
)



Epoch 1/4
 1/14 [=>............................] - ETA: 1:55:06 - loss: 1.0661 - accuracy: 0.2812

In [ ]:
# Evaluate on Test Data
test_texts = preprocess_texts(test_df["text"])
test_images = preprocess_images([f"images/{img}" for img in test_df["image_name"]])
test_labels = test_df["label"].values

results = model.evaluate(
    [test_images, test_texts["input_ids"], test_texts["attention_mask"]],
    test_labels
)
print(f"Test Accuracy: {results[1]}")